# Technical Analysis Of Intel

## Purpose
This phase of the project focuses on conducting an in-depth exploratory data analysis (EDA) of the selected stock. The goal is to extract valuable insights and explore key statistics that will lay the foundation for building a robust stock market forecasting system capable of providing actionable recommendations—Buy, Sell, or Hold.
### 1.0 Load Data and Calculate Key Statistics

## Potential stocks of interest 
    - INTC  
    - AAPL
    - INTC
    - MLTX
    - TSLA
    - HON


In [1]:
# Importing Libraries and Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import mplfinance as mpf
import plotly.graph_objects as go


company_df = pd.read_csv('data/sp500_companies.csv')
stock_df = pd.read_csv('data/sp500_stocks.csv')


In [ ]:
# Create lag columns
stock_df['close_lag1'] = stock_df['Close'].shift(1)
stock_df['close_lag2'] = stock_df['Close'].shift(2)
stock_df['close_lag3'] = stock_df['Close'].shift(3)
stock_df['close_lag4'] = stock_df['Close'].shift(5)
stock_df['close_lag5'] = stock_df['Close'].shift(10)

stock_df['volume_lag1'] = stock_df['Volume'].shift(1)
stock_df['volume_lag2'] = stock_df['Volume'].shift(2)
stock_df['volume_lag3'] = stock_df['Volume'].shift(3)
stock_df['volume_lag4'] = stock_df['Volume'].shift(5)
stock_df['volume_lag5'] = stock_df['Volume'].shift(10)



In [2]:
# Create new columns with Moving Averages and Standard Deviations

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

stock_df['MA_10'] = stock_df.groupby('Symbol')['Close'].rolling(window=10).mean().reset_index(level=0, drop=True)
stock_df['MA_20'] = stock_df.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(level=0, drop=True)
stock_df['MA_50'] = stock_df.groupby('Symbol')['Close'].rolling(window=50).mean().reset_index(level=0, drop=True)
stock_df['MA_200'] = stock_df.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(level=0, drop=True)
stock_df['std_10'] = stock_df.groupby('Symbol')['Close'].rolling(window=10).std().reset_index(level=0, drop=True)
stock_df['std_20'] = stock_df.groupby('Symbol')['Close'].rolling(window=20).std().reset_index(level=0, drop=True)
stock_df['std_50'] = stock_df.groupby('Symbol')['Close'].rolling(window=50).std().reset_index(level=0, drop=True)
stock_df['std_200'] = stock_df.groupby('Symbol')['Close'].rolling(window=200).std().reset_index(level=0, drop=True)


In [3]:
# Create new columns with Bollinger Bands for each Moving Average

stock_df['upper_band_10'] = stock_df['MA_10'] + (stock_df['std_10'] * 2)
stock_df['lower_band_10'] = stock_df['MA_10'] - (stock_df['std_10'] * 2)

stock_df['upper_band_20'] = stock_df['MA_20'] + (stock_df['std_20'] * 2)
stock_df['lower_band_20'] = stock_df['MA_20'] - (stock_df['std_20'] * 2)

stock_df['upper_band_50'] = stock_df['MA_50'] + (stock_df['std_50'] * 2)
stock_df['lower_band_50'] = stock_df['MA_50'] - (stock_df['std_50'] * 2)


stock_df['upper_band_200'] = stock_df['MA_200'] + (stock_df['std_200'] * 2)
stock_df['lower_band_200'] = stock_df['MA_200'] - (stock_df['std_200'] * 2)

In [4]:
# Create new columns Indicating Golden Cross and Death Cross

stock_df['Golden_Cross_Short'] = np.where((stock_df['MA_10'] > stock_df['MA_20']) & (stock_df['MA_10'].shift(1) <= stock_df['MA_20'].shift(1)), 1, 0)
stock_df['Golden_Cross_Medium'] = np.where((stock_df['MA_20'] > stock_df['MA_50']) & (stock_df['MA_20'].shift(1) <= stock_df['MA_50'].shift(1)), 1, 0)
stock_df['Golden_Cross_Long'] = np.where((stock_df['MA_50'] > stock_df['MA_200']) & (stock_df['MA_50'].shift(1) <= stock_df['MA_200'].shift(1)), 1, 0)

stock_df['Death_Cross_Short'] = np.where((stock_df['MA_10'] < stock_df['MA_20']) & (stock_df['MA_10'].shift(1) >= stock_df['MA_20'].shift(1)), 1, 0)
stock_df['Death_Cross_Medium'] = np.where((stock_df['MA_20'] < stock_df['MA_50']) & (stock_df['MA_20'].shift(1) >= stock_df['MA_50'].shift(1)), 1, 0)
stock_df['Death_Cross_Long'] = np.where((stock_df['MA_50'] < stock_df['MA_200']) & (stock_df['MA_50'].shift(1) >= stock_df['MA_200'].shift(1)), 1, 0)

In [5]:
# Create new columns for MACD, Signal Line and MACD Histogram

def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    # Calculate the short-term EMA
    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    
    # Calculate the long-term EMA
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    
    # Calculate the MACD line
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    
    # Calculate the Signal line
    df['Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    # Calculate the MACD histogram
    df['MACD_Hist'] = df['MACD'] - df['Signal']
    
    return df

stock_df = calculate_macd(stock_df)

In [6]:
# Create new columns for Average True Range (ATR) and True Range (TR)

stock_df['Previous_Close'] = stock_df['Close'].shift(1)

# True Range, Shows the volatility of the stock
stock_df['TR'] = stock_df.apply(
    lambda row: max(
        row['High'] - row['Low'],  # High - Low
        abs(row['High'] - row['Previous_Close']),  # |High - Previous Close|
        abs(row['Low'] - row['Previous_Close'])  # |Low - Previous Close|
    ), axis=1
)

# Average True Range, Shows the average volatility of the stock
stock_df['ATR'] = stock_df['TR'].rolling(window=10).mean()

In [7]:
# Create new columns for Relative Strength Index (RSI)

def calculate_rsi(df, window=10):
    # Calculate daily price changes
    delta = df['Close'].diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the average gain and average loss
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Add RSI column to the DataFrame
stock_df['RSI_10_Day'] = calculate_rsi(stock_df)

In [8]:
# Create new columns for 10 Day Rate of Change (ROC)
stock_df['10_Day_ROC'] = ((stock_df['Close'] - stock_df['Close'].shift(10)) / stock_df['Close'].shift(10)) * 100

In [9]:
# Create new columns for 10,20,50 day resistance and support levels
stock_df['Resistance_10_Day'] = stock_df['Close'].rolling(window=10).max()
stock_df['Support_10_Day'] = stock_df['Close'].rolling(window=10).min()
stock_df['Resistance_20_Day'] = stock_df['Close'].rolling(window=20).max()
stock_df['Support_20_Day'] = stock_df['Close'].rolling(window=20).min()
stock_df['Resistance_50_Day'] = stock_df['Close'].rolling(window=50).max()
stock_df['Support_50_Day'] = stock_df['Close'].rolling(window=50).min()

In [10]:
# Create new columns for 10,20,50 day Volume Indicators
stock_df['Volume_MA_10'] = stock_df['Volume'].rolling(window=10).mean()
stock_df['Volume_MA_20'] = stock_df['Volume'].rolling(window=20).mean()
stock_df['Volume_MA_50'] = stock_df['Volume'].rolling(window=50).mean()

In [11]:
def create_OBV(df):
    df['OBV'] = 0
    for i in range(1, len(df)):
        if df['Close'].iloc[i] > df['Close'].iloc[i - 1]:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1] + df['Volume'].iloc[i]
        elif df['Close'].iloc[i] < df['Close'].iloc[i - 1]:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1] - df['Volume'].iloc[i]
        else:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1]
    return df

stock_df = create_OBV(stock_df)

### Preview Data with Additional Columns

In [12]:

stock_df['Z-score'] = (stock_df['Close'] - stock_df['Close'].mean()) / stock_df['Close'].std()
stock_df['Daily Return'] = stock_df.groupby('Symbol')['Close'].pct_change(fill_method=None).reset_index(level=0, drop=True)
company_df.fillna(0, inplace=True)
stock_df.fillna(0, inplace=True)
stock_df

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,MA_10,MA_20,...,Resistance_20_Day,Support_20_Day,Resistance_50_Day,Support_50_Day,Volume_MA_10,Volume_MA_20,Volume_MA_50,OBV,Z-score,Daily Return
0,2010-01-04,MMM,44.016720,69.414719,69.774246,69.122070,69.473244,3640265.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0,-0.164107,0.000000
1,2010-01-05,MMM,43.741032,68.979935,69.590302,68.311035,69.230766,3405012.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,-3405012,-0.166190,-0.006264
2,2010-01-06,MMM,44.361351,69.958191,70.735786,69.824417,70.133781,6301126.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,2896114,-0.161504,0.014182
3,2010-01-07,MMM,44.393173,70.008362,70.033447,68.662209,69.665550,5346240.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,8242354,-0.161263,0.000717
4,2010-01-08,MMM,44.705986,70.501671,70.501671,69.648827,69.974915,4073337.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,12315691,-0.158900,0.007046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864113,2024-09-18,ZTS,196.479996,196.479996,198.389999,194.369995,196.800003,1993800.0,191.714000,187.348000,...,196.479996,180.899994,196.479996,173.490005,1888330.0,1684680.00,1872634.00,353530370169,0.444592,0.003934
1864114,2024-09-19,ZTS,193.240005,193.240005,200.330002,191.309998,198.229996,2463100.0,192.379001,187.865001,...,196.479996,180.899994,196.479996,174.820007,1988070.0,1745495.00,1882476.00,353527907069,0.429071,-0.016490
1864115,2024-09-20,ZTS,192.639999,192.639999,194.550003,191.070007,192.940002,3520700.0,192.720001,188.388501,...,196.479996,180.899994,196.479996,174.820007,2069130.0,1864740.00,1913092.00,353524386369,0.426197,-0.003105
1864116,2024-09-23,ZTS,193.589996,193.589996,193.660004,191.080002,192.809998,1315700.0,192.906001,189.023001,...,196.479996,181.559998,196.479996,174.820007,1935430.0,1867405.00,1905950.00,353525702069,0.430748,0.004931
